In [1]:
import pandas as pd

<span style="color:grey"> Leemos las distintas hojas y creamos los distintos DataFrames.


In [2]:
#Leemos la hoja de los países y renombramos las columnas
df_argentina = pd.read_excel('Prueba_Data_Engineer.xlsx', sheet_name='Argentina')
df_argentina.rename(columns={'Identificador': 'Identificador', 'Ventas': 'Ventas', 'Divisas': 'Divisas_Ventas'}, inplace=True)
df_espana = pd.read_excel('Prueba_Data_Engineer.xlsx', sheet_name='España')
df_espana.rename(columns={'Identificador': 'Identificador', 'VTA': 'Ventas', 'Divisas': 'Divisas_Ventas'}, inplace=True)
df_mexico = pd.read_excel('Prueba_Data_Engineer.xlsx', sheet_name='Mexico')
df_mexico.rename(columns={'Identificador': 'Identificador', 'Ventas': 'Ventas', 'Divisas': 'Divisas_Ventas'}, inplace=True)

#Ahora leemos la hoja central y creamos la columna Identificador como combinación de las columnas país, id_empleado e id_departamento para poder realizar los joins correctamente.
df_central = pd.read_excel('Prueba_Data_Engineer.xlsx', sheet_name='Fichero_central')
df_central['Identificador'] = df_central['Pais'].astype(str) + '0000' + df_central['Id departamento'].astype(str) + '0' + df_central['ID Empleado'].astype(str)


<span style="color:grey"> Ahora mergeamos todos los DataFrames y creamos nuestra hoja principal según los requisitos descritos.


In [3]:
# Unimos todos los df y creamos nuestro df principal
df_final = pd.merge(df_central, df_espana, on='Identificador', how='left')
df_final = pd.merge(df_final, df_argentina, on='Identificador', how='left')
df_final = pd.merge(df_final, df_mexico, on='Identificador', how='left')

#Mergeamos ventas
df_final['Ventas'] = df_final['Ventas'].fillna(df_final['Ventas_x']).fillna(df_final['Ventas_y'])
df_final = df_final.drop(['Ventas_x', 'Ventas_y'], axis=1)

#Mergeamos divisas
df_final['Divisas_Ventas'] = df_final['Divisas_Ventas'].fillna(df_final['Divisas_Ventas_x']).fillna(df_final['Divisas_Ventas_y'])
df_final = df_final.drop(['Divisas_Ventas_x', 'Divisas_Ventas_y'], axis=1)

#Tipos de cambio:
tipo_cambio = {
    'Pesos Mexicanos': 0.055,
    'Dolares': 0.93,  # Corregido 'Dolar' a 'Dolares'
    'Pesos Argentinos': 0.0013}

# Función para aplicar el tipo de cambio según la divisa
def convertir_a_euro(row, valor, divisa):
    if divisa == 'Euros':
        return valor
    elif divisa in tipo_cambio:
        tipo_cambio_divisa = tipo_cambio[divisa]
        return valor * tipo_cambio_divisa
    else:
        return None  # Devolver None si la divisa no está en el tipo de cambio

# Función para aplicar el tipo de cambio a las columnas Ventas y Divisas_Ventas
def aplicar_tipo_cambio_ventas(row):
    return convertir_a_euro(row, row['Ventas'], row['Divisas_Ventas'])

# Función para aplicar el tipo de cambio a las columnas Coste y Divisas
def aplicar_tipo_cambio_coste(row):
    return convertir_a_euro(row, row['Coste'], row['Divisas'])

# Creamos la nueva columna 'Ventas_euros' aplicando el tipo de cambio
df_final['Ventas_euros'] = df_final.apply(aplicar_tipo_cambio_ventas, axis=1)

# Creamos la nueva columna 'Coste_euros' aplicando el tipo de cambio
df_final['Coste_euros'] = df_final.apply(aplicar_tipo_cambio_coste, axis=1)

#Creamos la columna margen y la columna %margen
df_final['Margen'] = (df_final['Ventas_euros'] - df_final['Coste_euros']) / df_final['Ventas_euros']
df_final['%Margen'] = (df_final['Ventas_euros'] - df_final['Coste_euros'])*100 / df_final['Ventas_euros']

<span style="color:grey"> Ahora calculamos el total de ventas y costes totales en Euros.


In [4]:
#TOTAL VENTAS Y COSTES:
total_ventas = df_final['Ventas_euros'].sum()
total_coste = df_final['Coste_euros'].sum()

print("El total de ventas en € es " + str(total_ventas))
print("El coste total en € es " + str(total_coste))

El total de ventas en € es 17511833.708
El coste total en € es 882766.6199999999


<span style="color:grey"> A continuación calculamos el total de ventas y costes por país, tanto en moneda local como moneda euro. Para ello vamos a transformar todos los valores en dólares de los pesos mexicanos y argentinos.

In [5]:
tipo_cambio_pais = {
    'MX': 0.06,
    'AR': 0.0011, 
    'ES': 1.0753   
}
def convertir_divisas(df, tipo_cambio_pais):
    for index, row in df.iterrows():
        pais = row['Pais']
        divisa = row['Divisas']
        if divisa == 'Dolares':
            tipo_cambio = tipo_cambio_pais.get(pais)
            if tipo_cambio:
                df.at[index, 'Coste'] *= tipo_cambio
                df.at[index, 'Ventas'] *= tipo_cambio
                if pais == 'AR':
                    df.at[index, 'Divisas'] = 'Pesos Argentinos'
                    df.at[index, 'Divisas_Ventas'] = 'Pesos Argentinos'
                elif pais == 'MX':
                    df.at[index, 'Divisas'] = 'Pesos Mexicanos'
                    df.at[index, 'Divisas_Ventas'] = 'Pesos Mexicanos'
        elif divisa == 'Divisa local':
            if pais == 'AR':
                df.at[index, 'Divisas'] = 'Pesos Argentinos'
                df.at[index, 'Divisas_Ventas'] = 'Pesos Argentinos'
            elif pais == 'MX':
                df.at[index, 'Divisas'] = 'Pesos Mexicanos'
                df.at[index, 'Divisas_Ventas'] = 'Pesos Mexicanos'
    return df

df_ajustado = convertir_divisas(df_final, tipo_cambio_pais)
df_ventas_costes_dos_divisas = df_ajustado.groupby('Pais').agg({
    'Coste': 'sum',
    'Ventas': 'sum',
    'Ventas_euros': 'sum',
    'Coste_euros': 'sum'
}).reset_index()

# Renombrando las columnas
df_ventas_costes_dos_divisas.columns = ['Pais', 'Coste', 'Ventas', 'Ventas_euros', 'Coste_euros']
df_ventas_costes_dos_divisas

,Pais,Coste,Ventas,Ventas_euros,Coste_euros
0,AR,84865425,9.743159e+09,1.266935e+07,131826.62
1,ES,510230,5.030946e+05,5.030946e+05,510230.00
2,MX,1141520,6.888418e+07,4.339390e+06,240710.00


<span style="color:grey"> Por último, calculamos el total de ventas y coste por departamento descriptivo, en moneda del país de origen y en moneda euro. Para ello vamos a renombrar todos los valores de Recursos Humanos y R.R.H.H a RRHH y con Equipo comercial y Comerciales renombrarlos a Comercial.

In [6]:
df_ajustado['Nombre departamento'] = df_ajustado['Nombre departamento'].replace({
    'Recursos Humanos': 'RRHH',
    'R.R.H.H': 'RRHH',
    'Equipo comercial': 'Comercial',
    'Comerciales': 'Comercial'
})
df_ventas_costes_dos_divisas = df_ajustado.groupby('Nombre departamento').agg({
    'Coste': 'sum',
    'Ventas': 'sum',
    'Ventas_euros': 'sum',
    'Coste_euros': 'sum'
}).reset_index()

# Renombrando las columnas
df_ventas_costes_dos_divisas.columns = ['Nombre departamento', 'Coste', 'Ventas', 'Ventas_euros', 'Coste_euros']
df_ventas_costes_dos_divisas

,Nombre departamento,Coste,Ventas,Ventas_euros,Coste_euros
0,Comercial,53851025,6.163690e+09,8826817.908,474866.32
1,Facturación,60000,0.000000e+00,0.000,60000.00
2,RRHH,32606150,3.648856e+09,8685015.800,347900.30
